# Scenario
We imagine we are the analysts for a Dota 2 team called "Team Secret". 

In [186]:
import ipywidgets as widgets
import pickle
import pandas as pd
from IPython.display import clear_output

class DotaTeam:
    def __init__(self,team_id,name):
        self.team_id = team_id
        self.name = name
    def __str__(self):
        return str(self.__dict__)
    def __repr__(self):
        return str(self)
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__,
            sort_keys=True, indent=4)
    
class PickBan:
    def __init__(self,hero,win,loss):
        self.hero = hero
        self.win = win
        self.loss = loss
    def __str__(self):
        return str(self.__dict__)
    def __repr__(self):
        return str(self)
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__,
            sort_keys=True, indent=4)

def load_obj(name):
    with open('./Data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [ ]:
# Load data from Data_Collection
data = load_obj('matchdata_19_20')

In [329]:
# Load heroes meta data
df_hero_list = pd.read_json('Data/heroes.json').T
df_hero_list = df_hero_list[['id','localized_name']]
df_hero_list.set_index("id", inplace = True)
df_hero_list.columns = ['Hero Name']

In [30]:
teams_list = []

for match in data:
    if 'radiant_team' in match:
        if match['radiant_team']['name'] != '':
            if not any(team.team_id == match['radiant_team']['team_id'] for team in teams_list):
                teams_list.append(DotaTeam(match['radiant_team']['team_id'], match['radiant_team']['name']))
    if 'dire_team' in match:
        if match['dire_team']['name'] != '':
            if not any(team.team_id == match['dire_team']['team_id'] for team in teams_list):
                teams_list.append(DotaTeam(match['dire_team']['team_id'], match['dire_team']['name']))

In [33]:
print('Number of unique teams: ', len(teams_list))

Number of unique teams:  2011


In [427]:
team_secret_matches = []
for match in data:
    if 'radiant_team' in match:
        if match['radiant_team']['team_id'] == 15:
            team_secret_matches.append(match)
    if 'dire_team' in match:
        if match['dire_team']['team_id'] == 15:
            team_secret_matches.append(match)
            
for match in team_secret_matches:
    team_secret_mean_match_time += match['duration']
    
team_secret_mean_match_time = team_secret_mean_match_time/len(team_secret_matches)
print('MEAN: ', round(team_secret_mean_match_time/60, 1))
print('Number games: ', len(team_secret_matches))


MEAN:  34.9
Number games:  298


In [433]:
def on_button_clicked(b):
    with output:
        clear_output(wait=False)

        # ========================================================
        # = Analyze Team Secret games
        # ========================================================
        team_secret_matches = []
        for match in data:
            if 'radiant_team' in match:
                if match['radiant_team']['team_id'] == 1838315:
                    team_secret_matches.append(match)
            if 'dire_team' in match:
                if match['dire_team']['team_id'] == 1838315:
                    team_secret_matches.append(match)

        # Calculate key values
        team_secret_picked_heroes = []
        team_secret_banned_heroes = []
        team_secret_mean_match_time = 0

        for match in team_secret_matches:
            team_secret_mean_match_time += match['duration']
            # If they are or radiant
            if 'radiant_team' in match:
                if match['radiant_team']['team_id'] == 1838315 and match['picks_bans'] is not None:
                    for pick_ban in match['picks_bans']:
                        # Add to picked heroes
                        if pick_ban['is_pick']:
                            if pick_ban['team'] == 0:
                                if match['radiant_win']:
                                    # Win with picked hero
                                    team_secret_picked_heroes.append(PickBan(pick_ban['hero_id'], 1, 0))
                                else:
                                    # Loss with picked hero
                                    team_secret_picked_heroes.append(PickBan(pick_ban['hero_id'], 0, 1))
                        # Add to banned heroes
                        else:
                            if pick_ban['team'] == 0:
                                if match['radiant_win']:
                                    # Win with banned hero
                                    team_secret_banned_heroes.append(PickBan(pick_ban['hero_id'], 1, 0))
                                else:
                                    # Loss with banned hero
                                    team_secret_banned_heroes.append(PickBan(pick_ban['hero_id'], 0, 1))
            # If they are or dire
            if 'dire_team' in match:
                if match['dire_team']['team_id'] == 1838315 and match['picks_bans'] is not None:
                    for pick_ban in match['picks_bans']:
                        # Add to picked heroes
                        if pick_ban['is_pick']:
                            if pick_ban['team'] == 1:
                                if match['radiant_win']:
                                    # Loss with picked hero
                                    team_secret_picked_heroes.append(PickBan(pick_ban['hero_id'], 0, 1))
                                else:
                                    # Win with picked hero
                                    team_secret_picked_heroes.append(PickBan(pick_ban['hero_id'], 1, 0))
                        # Add to banned heroes            
                        else:
                            if pick_ban['team'] == 1:
                                if match['radiant_win']:
                                    # Loss with picked hero
                                    team_secret_banned_heroes.append(PickBan(pick_ban['hero_id'], 0, 1))
                                else:
                                    # Win with picked hero
                                    team_secret_banned_heroes.append(PickBan(pick_ban['hero_id'], 1, 0))

        # Average game length
        team_secret_mean_match_time = team_secret_mean_match_time/len(team_secret_matches)

        # Count picked heroes
        df_secret_picked_heroes = pd.DataFrame(team_secret_picked_heroes)
        secret_picks = pd.DataFrame(columns=['hero_id', 'Wins', 'Losses'])

        for index, row in df_secret_picked_heroes.iterrows():
            secret_picks = secret_picks.append({'hero_id': row[0].hero, 'Wins': row[0].win, 'Losses': row[0].loss}, ignore_index=True)
        secret_picks = secret_picks.groupby(['hero_id']).sum()
        secret_picks['Number Picks'] = secret_picks['Wins'] + secret_picks['Losses']
        secret_picks['Winrate'] = secret_picks['Wins'] / secret_picks['Number Picks']
        secret_picks = secret_picks.join(df_hero_list, how = "outer")
        secret_picks = secret_picks.fillna(0)

        secret_heroes_picked_sorted = secret_picks.sort_values(by=['Number Picks'])
        secret_picked_smallest = secret_heroes_picked_sorted.nsmallest(10, 'Number Picks')
        secret_picked_greatest = secret_heroes_picked_sorted.nlargest(10, 'Number Picks')

        secret_heroes_picked_sorted = secret_picks.sort_values(by=['Winrate', 'Number Picks'], ascending=[False, False])
        secret_most_successful = secret_heroes_picked_sorted.nlargest(10, 'Winrate')
        
        # Count banned heroes
        df_secret_banned_heroes = pd.DataFrame(team_secret_banned_heroes)
        secret_bans = pd.DataFrame(columns=['hero_id', 'Wins', 'Losses'])

        for index, row in df_secret_banned_heroes.iterrows():
            secret_bans = secret_bans.append({'hero_id': row[0].hero, 'Wins': row[0].win, 'Losses': row[0].loss}, ignore_index=True)
        secret_bans = secret_bans.groupby(['hero_id']).sum()
        secret_bans['Number Bans'] = secret_bans['Wins'] + secret_bans['Losses']
        secret_bans['Winrate'] = secret_bans['Wins'] / secret_bans['Number Bans']
        secret_bans = secret_bans.join(df_hero_list, how = "outer")
        secret_bans = secret_bans.fillna(0)    

        secret_heroes_banned_sorted = secret_bans.sort_values(by=['Number Bans'])
        secret_banned_smallest = secret_heroes_banned_sorted.nsmallest(10, 'Number Bans')
        secret_banned_greatest = secret_heroes_banned_sorted.nlargest(10, 'Number Bans')

        print('\n***********************************************')
        print('************** Team Secret Stats **************')
        print('*********************************************** \n')
        print('Team Secret game time in minutes: ', round(team_secret_mean_match_time/60, 1))
        print('\n Most picked heroes:')
        print(secret_picked_greatest[['Hero Name', 'Winrate', 'Wins', 'Losses', 'Number Picks']].to_string(index=False))
        print('\n Least picked heroes:')
        print(secret_picked_smallest[['Hero Name', 'Winrate', 'Wins', 'Losses', 'Number Picks']].to_string(index=False))
        print('\n Most banned heroes:')
        print(secret_banned_greatest[['Hero Name', 'Winrate', 'Wins', 'Losses', 'Number Bans']].to_string(index=False))
        print('\n Least banned heroes:')
        print(secret_banned_smallest[['Hero Name', 'Winrate', 'Wins', 'Losses', 'Number Bans']].to_string(index=False))
        print('\n Most successful heroes:')
        print(secret_most_successful[['Hero Name', 'Winrate', 'Wins', 'Losses', 'Number Picks']].to_string(index=False))
        
        # ========================================================
        # = Analyze opponent
        # ========================================================
        team_to_analyze = 0

        for team in teams_list:
            if team.name == team_to_analyze_input.value:
                team_to_analyze = team.team_id
        if team_to_analyze == 0:
            print('Team not found!')

        # Filter opponent matches
        opponent_matches = []

        for match in data:
            if 'radiant_team' in match:
                if match['radiant_team']['team_id'] == team_to_analyze:
                    opponent_matches.append(match)
            if 'dire_team' in match:
                if match['dire_team']['team_id'] == team_to_analyze:
                    opponent_matches.append(match)

        # Calculate key values
        opponent_picked_heroes = []
        opponent_banned_heroes = []
        opponent_mean_match_time = 0

        for match in opponent_matches:
            opponent_mean_match_time += match['duration']
            # If they are on radiant
            if 'radiant_team' in match:
                if match['radiant_team']['team_id'] == team_to_analyze and match['picks_bans'] is not None:
                    for pick_ban in match['picks_bans']:
                        # Add to picked heroes
                        if pick_ban['is_pick']:
                            if pick_ban['team'] == 0:
                                if match['radiant_win']:
                                    # Win with picked hero
                                    opponent_picked_heroes.append(PickBan(pick_ban['hero_id'], 1, 0))
                                else:
                                    # Loss with picked hero
                                    opponent_picked_heroes.append(PickBan(pick_ban['hero_id'], 0, 1))
                        # Add to banned heroes
                        else:
                            if pick_ban['team'] == 0:
                                if match['radiant_win']:
                                    # Win with banned hero
                                    opponent_banned_heroes.append(PickBan(pick_ban['hero_id'], 1, 0))
                                else:
                                    # Loss with banned hero
                                    opponent_banned_heroes.append(PickBan(pick_ban['hero_id'], 0, 1))
            # If they are on dire
            if 'dire_team' in match:
                if match['dire_team']['team_id'] == team_to_analyze and match['picks_bans'] is not None:
                    for pick_ban in match['picks_bans']:
                        # Add to picked heroes
                        if pick_ban['is_pick']:
                            if pick_ban['team'] == 1:
                                if match['radiant_win']:
                                    # Loss with picked hero
                                    opponent_picked_heroes.append(PickBan(pick_ban['hero_id'], 0, 1))
                                else:
                                    # Win with picked hero
                                    opponent_picked_heroes.append(PickBan(pick_ban['hero_id'], 1, 0))
                        # Add to banned heroes            
                        else:
                            if pick_ban['team'] == 1:
                                if match['radiant_win']:
                                    # Loss with picked hero
                                    opponent_banned_heroes.append(PickBan(pick_ban['hero_id'], 0, 1))
                                else:
                                    # Win with picked hero
                                    opponent_banned_heroes.append(PickBan(pick_ban['hero_id'], 1, 0))

        if len(opponent_matches) != 0 :
            print("NUMBER OF OPPONENT MATCHES: ", len(opponent_matches))
            # Average game length
            opponent_mean_match_time = opponent_mean_match_time/len(opponent_matches)


            # Count picked heroes
            df_opponent_picked_heroes = pd.DataFrame(opponent_picked_heroes)
            opponent_picks = pd.DataFrame(columns=['hero_id', 'Wins', 'Losses'])

            for index, row in df_opponent_picked_heroes.iterrows():
                opponent_picks = opponent_picks.append({'hero_id': row[0].hero, 'Wins': row[0].win, 'Losses': row[0].loss}, ignore_index=True)
            opponent_picks = opponent_picks.groupby(['hero_id']).sum()
            opponent_picks['Number Picks'] = opponent_picks['Wins'] + opponent_picks['Losses']
            opponent_picks['Winrate'] = opponent_picks['Wins'] / opponent_picks['Number Picks']
            opponent_picks = opponent_picks.join(df_hero_list, how = "outer")
            opponent_picks = opponent_picks.fillna(0)

            opponent_heroes_picked_sorted = opponent_picks.sort_values(by=['Number Picks'])
            opponent_picked_smallest = opponent_heroes_picked_sorted.nsmallest(10, 'Number Picks')
            opponent_picked_greatest = opponent_heroes_picked_sorted.nlargest(10, 'Number Picks')

            opponent_heroes_picked_sorted = opponent_picks.sort_values(by=['Winrate', 'Number Picks'], ascending=[False, False])
            opponent_most_successful = opponent_heroes_picked_sorted.nlargest(10, 'Winrate')
        
            # Count banned heroes
            df_opponent_banned_heroes = pd.DataFrame(opponent_banned_heroes)
            opponent_bans = pd.DataFrame(columns=['hero_id', 'Wins', 'Losses'])

            for index, row in df_opponent_banned_heroes.iterrows():
                opponent_bans = opponent_bans.append({'hero_id': row[0].hero, 'Wins': row[0].win, 'Losses': row[0].loss}, ignore_index=True)
            opponent_bans = opponent_bans.groupby(['hero_id']).sum()
            opponent_bans['Number Bans'] = opponent_bans['Wins'] + opponent_bans['Losses']
            opponent_bans['Winrate'] = opponent_bans['Wins'] / opponent_bans['Number Bans']
            opponent_bans = opponent_bans.join(df_hero_list, how = "outer")
            opponent_bans = opponent_bans.fillna(0)    

            opponent_heroes_banned_sorted = opponent_bans.sort_values(by=['Number Bans'])
            opponent_banned_smallest = opponent_heroes_banned_sorted.nsmallest(10, 'Number Bans')
            opponent_banned_greatest = opponent_heroes_banned_sorted.nlargest(10, 'Number Bans')

            print('\n\n\n***********************************************')
            print('************** ' + team_to_analyze_input.value + ' Stats **************')
            print('*********************************************** \n')
            print(team_to_analyze_input.value + ' game time in minutes: ', round(opponent_mean_match_time/60, 1))
            print('\n Most picked heroes:')
            print(opponent_picked_greatest[['Hero Name', 'Winrate', 'Wins', 'Losses', 'Number Picks']].to_string(index=False))
            print('\n Least picked heroes:')
            print(opponent_picked_smallest[['Hero Name', 'Winrate', 'Wins', 'Losses', 'Number Picks']].to_string(index=False))
            print('\n Most banned heroes:')
            print(opponent_banned_greatest[['Hero Name', 'Winrate', 'Wins', 'Losses', 'Number Bans']].to_string(index=False))
            print('\n Least banned heroes:')
            print(opponent_banned_smallest[['Hero Name', 'Winrate', 'Wins', 'Losses', 'Number Bans']].to_string(index=False))
            print('\n Most successful heroes:')
            print(opponent_most_successful[['Hero Name', 'Winrate', 'Wins', 'Losses', 'Number Picks']].to_string(index=False))
        
            
            
button = widgets.Button(description="Analyze!")
output = widgets.Output()


button.on_click(on_button_clicked)

team_to_analyze_input = widgets.Text(
    value='Nigma',
    placeholder='Type something',
    description='Opponent:',
    disabled=False
)

In [434]:
display(team_to_analyze_input)

display(button, output)


Text(value='Nigma', description='Opponent:', placeholder='Type something')

Button(description='Analyze!', style=ButtonStyle())

Output()